In [2]:
import resources as r
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML, display
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from tqdm.notebook import tqdm_notebook
from copy import deepcopy
IPythonConsole.ipython_useSVG=True

layout = widgets.Layout(width='max-content')

display(HTML('''<style>
    .widget-label { min-width: 30ex !important; }
    .widget-inline-hbox { min-width: 50ex !important; }
    .widget-button { min-width: max-content }
</style>'''))

fluorophores, solvents, methods = r.fluorophores_solvents_methods()

def extractIndices(df):
    levels = len(df.index[0])
    indexLists = [[] for i in range(levels)]
    for row in df.index:
        for count, level in enumerate(row):
            if level not in indexLists[count]:
                indexLists[count] += [level] 
    return [df.columns.to_list()] + indexLists

with r.statusLoad('progress') as df:
    metaJobList = extractIndices(df)[3]

# DF Creation and modification
def make_multindex(cols:list[str], iterables:list, levelLabels:list[str], dfName:str, initValue, astype:type) -> None:
    rows = 1
    for inner in iterables:
        rows = rows * len(inner)
    index = pd.MultiIndex.from_product(iterables, names=levelLabels)
    df = pd.DataFrame(np.full((rows, len(cols)), initValue), columns=cols, index=index)
    df.astype(astype)
    df.to_pickle(f'resources/databases/fluorophores-ds/{dfName}')

def add_metajob() -> None:
    metajob = metajob_widg.value
    out.clear_output()
    with out:
        for db in ['progress', 'comp-freq', 'comp-em', 'comp-ex', 'comp-casscf']:
            with r.statusLoad(df=db) as df:
                if db == 'progress': 
                    solvents, fluorophores, states, metajobs = extractIndices(df)
                    for fluorophores in fluorophores:
                        for state in states:
                            df.loc[(fluorophores, state, metajob), :] = None
                            df.sort_index(inplace=True)
                else:
                    print(db)
                    solvents, fluorophores, states, metajobs, properties = extractIndices(df)
                    if metajob.job == metajobs[0].job:
                        for solvent in solvents:
                            for fluorophore in fluorophores:
                                for state in states:
                                    for prop in properties:
                                        df.loc[(fluorophore, state, metajob, prop), solvent] = 0.0
                                        df.sort_index(inplace=True)
    if displayDF_widg.value:
        prettyDisplay(full=True)
        return

def rem_metajob():
    metajob = metajob_widg.value
    out.clear_output()
    with out:
        for ds in ['progress', 'comp-freq', 'comp-em', 'comp-ex', 'comp-casscf']:
            with r.statusLoad(df=ds) as df:
                try:
                    df = df.drop(index=metajob, level=2, inplace=True)
                except AttributeError:
                    pass
                except KeyError:
                    pass
    if displayDF_widg.value:
        prettyDisplay(full=True)
    return

def add_fluorophore():
    global dbList
    fluorophore = fluorophore_widg.value
    out.clear_output()
    with out:
        for db in dbList:
            initval = 0.0 if db in ['dataset', 'comp-freq', 'comp-em', 'comp-ex', 'comp-casscf'] else None
            with r.statusLoad(df=db) as df:
                if db == 'progress': 
                    solvents, fluorophores, states, metajobs = extractIndices(df)
                    for state in states:
                        for metajob in metajobs:
                            df.loc[(fluorophore, state, metajob), :] = initval
                            df.sort_index(inplace=True)
                elif db in ['comp-freq', 'comp-em', 'comp-ex', 'comp-casscf']:
                    solvents, fluorophores, states, metajobs, properties = extractIndices(df)
                    for state in states:
                        for metajob in metajobs:
                            for prop in properties:
                                df.loc[(fluorophore, state, metajob, prop), :] = initval
                                df.sort_index(inplace=True)
                else: 
                    solvents, fluorophores, inners = extractIndices(df)
                    for inner in inners:
                        df.loc[(fluorophore, inner), :] = initval
                        df.sort_index(inplace=True)
    if displayDF_widg.value:
        prettyDisplay(full=True)
    return

def rem_fluorophore():
    global dbList
    fluorophore = fluorophore_widg.value
    out.clear_output()
    with out:
        for db in dbList:
            with r.statusLoad(df=db) as df:
                try:
                    df = df.drop(index=fluorophore, level=0, inplace=True)
                except AttributeError:
                    pass
    if displayDF_widg.value:
        prettyDisplay(full=True)
    return

def add_solvent():
    solvent = solvent_widg.value
    out.clear_output()
    with out:
        for db in dbList:
            initval = 0.0 if db in ['dataset', 'comp-freq', 'comp-em', 'comp-ex', 'comp-casscf'] else None
            with r.statusLoad(df=db) as df:
                if db == 'progress': 
                    solvents, fluorophores, states, metajobs = extractIndices(df)
                    for fluorophore in fluorophores:
                        for state in states:
                            for metajob in metajobs:
                                df.loc[(fluorophore, state, metajob), solvent] = initval
                                df.sort_index(inplace=True)
                elif db in ['comp-freq', 'comp-em', 'comp-ex', 'comp-casscf']:
                    solvents, fluorophores, states, metajobs, properties = extractIndices(df)
                    for fluorophore in fluorophores:
                        for state in states:
                            for metajob in metajobs:
                                for prop in properties:
                                    df.loc[(fluorophore, state, metajob, prop), solvent] = initval
                                    df.sort_index(inplace=True)
                else: 
                    solvents, fluorophores, inners = extractIndices(df)
                    for fluorophore in fluorophores:
                        for inner in inners:
                            df.loc[(fluorophore, inner), solvent] = initval
                            df.sort_index(inplace=True)
    if displayDF_widg.value:
        prettyDisplay(full=True)
    return

def rem_solvent():
    solvent = solvent_widg.value
    out.clear_output()
    with out:
        for db in dbList:
            with r.statusLoad(df=db) as df:
                try:
                    df = df.drop(solvent, axis=1, inplace=True)
                except AttributeError:
                    pass
    if displayDF_widg.value:
        prettyDisplay(full=True)
    return

# Data Pullers
def check_progress(mon:r.monarchHandler) -> None:
    out.clear_output()
    global stateList
    with out:
        with r.statusLoad(df='progress') as df:
            solvents, fluorophores, states, metajobs = extractIndices(df)
            for metajob in tqdm_notebook(list(jobs_widg.value), desc="Total", leave=False):
              if metajob in list(jobs_widg.value):
                    states = deepcopy(stateList)
                    if not metajob.gs:
                        try: 
                            states.remove(r.States.s0)
                            print(f'{r.States.s0} not logical with {metajob}')
                        except ValueError: pass
                    if not metajob.es:
                        try: 
                            states.remove(r.States.s1)
                            print(f'{r.States.s1} not logical with {metajob}')
                        except ValueError: pass
                        try: 
                            states.remove(r.States.s2)
                            print(f'{r.States.s2} not logical with {metajob}')
                        except ValueError: pass
                    for state in tqdm_notebook(states, desc=f"{metajob}"):
                        if state == r.States.s2:
                            fluorophores = [fluorophore for fluorophore in r.Fluorophores if (fluorophore.root == r.States.s2 and bool(fluorophore) == True)]
                        for fluorophore in tqdm_notebook(fluorophores, desc=f"{state}", leave=False):
                            solventList = [r.Solvents.gas] if metajob.gasonly else solvents + [r.Solvents.gas]
                            for solvent in tqdm_notebook(solventList, desc=f"{fluorophore}", leave=False):
                                if (fluorophore.gas) and (not fluorophore.revised) and (solvent != r.Solvents.gas):
                                    # print(f'{fluorophore} is gas-only and cannot be used with {solvent}')
                                    pass
                                else:
                                    status = df.at[(fluorophore, state, metajob), solvent]
                                    if (status != r.Status.finished) or fullCheck_widg.value:
                                        job = r.Job.from_MetaJob(metajob, fluorophore, solvent, state)
                                        status = mon.checkJobStatus(job)
                                        if status != None:
                                            df.at[(fluorophore, state, metajob), solvent] = status
    if displayDF_widg.value:
        prettyDisplay()

# Visualisers
def visualise_ds() -> None:
    out.clear_output()
    with out:
        print('Dataset')
        mols = []
        nameList = []
        for fluorophore in r.Fluorophores:
            if fluorophore.revised:
                nameList += [fluorophore.fluorophore]
                mols += [Chem.MolFromSmiles(fluorophore.smiles)]
        display(Draw.MolsToGridImage(mols, legends=nameList, subImgSize=(400, 400)))
        print('\nGas Phase')
        mols = []
        nameList = []
        for fluorophore in r.Fluorophores:
            if fluorophore.gas:
                nameList += [fluorophore.fluorophore]
                mols += [Chem.MolFromSmiles(fluorophore.smiles)]
        display(Draw.MolsToGridImage(mols, legends=nameList, subImgSize=(400, 400)))
        print('\nReference Species')
        mols = []
        nameList = []
        for fluorophore in r.Fluorophores:
            if fluorophore.ref:
                nameList += [fluorophore.fluorophore]
                mols += [Chem.MolFromSmiles(fluorophore.smiles)]
        display(Draw.MolsToGridImage(mols, legends=nameList, subImgSize=(400, 400)))

def prettyDisplay(full=False) -> None:
    global stateList
    with out:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            if spectra_widg.value:
                with r.statusLoad('spectra') as df:
                    print('Spectra Database')
                    display(df.notnull().style.applymap(lambda x: 'color : blue' if x == True  else 'color : red'))
                    print('\n')

            if energy_widg.value:
                with r.statusLoad('dataset') as df:
                    display(df.style.applymap(lambda x: 'color : blue' if x != None  else 'color : red').format(precision=3))
                    print('\n')

            if comp_freq_widg.value:
                with r.statusLoad('comp-freq') as df:
                    display(df.style.applymap(lambda x: 'color : blue' if x != 0.0  else 'color : red').format(precision=3))
                    print('\n')

            if comp_ex_widg.value:
                with r.statusLoad('comp-ex') as df:
                    display(df.style.applymap(lambda x: 'color : blue' if x != 0.0  else 'color : red').format(precision=3))
                    print('\n')

            if comp_em_widg.value:
                with r.statusLoad('comp-em') as df:
                    display(df.style.applymap(lambda x: 'color : blue' if x != 0.0  else 'color : red').format(precision=3))
                    print('\n')

            if comp_casscf_widg.value:
                with r.statusLoad('comp-casscf') as df:
                    display(df.style.applymap(lambda x: 'color : blue' if x != 0.0  else 'color : red').format(precision=3))
                    print('\n')

            if full:
                with r.statusLoad(f'progress') as df:
                        print(f'Full Dataset')
                        display(df.style.applymap(lambda x: 'color : red'     if x == r.Status.failed     else '')
                                        .applymap(lambda x: 'color : teal'    if x == r.Status.finished   else '')
                                        .applymap(lambda x: 'color : orange'  if x == r.Status.running    else '')
                                        .applymap(lambda x: 'color : blue'    if x == r.Status.queued     else '')
                                        .applymap(lambda x: 'color : purple'  if x == r.Status.timed_out  else '')
                                        .applymap(lambda x: 'color : grey'   if x == None                else ''))
                        print('\n')
            elif progress_widg.value:
                with r.statusLoad(f'progress') as df:
                    for metajob in list(jobs_widg.value):
                        states = deepcopy(stateList)
                        if not metajob.gs:
                            try: 
                                states.remove(r.States.s0)
                            except ValueError: pass
                        if not metajob.es:
                            try: 
                                states.remove(r.States.s1)
                            except ValueError: pass
                            try: 
                                states.remove(r.States.s2)
                            except ValueError: pass

                        for state in states:
                                print(f'Progress')
                
                                print(f'{metajob}: {state}')
                                display(df.loc[(slice(None), state, metajob)].style
                                                .applymap(lambda x: 'color : red'     if x == r.Status.failed     else '')
                                                .applymap(lambda x: 'color : teal'    if x == r.Status.finished   else '')
                                                .applymap(lambda x: 'color : orange'  if x == r.Status.running    else '')
                                                .applymap(lambda x: 'color : blue'    if x == r.Status.queued     else '')
                                                .applymap(lambda x: 'color : purple'  if x == r.Status.timed_out  else '')
                                                .applymap(lambda x: 'color : grey'    if x == None                else ''))
                                print('\n')


# Callable
def resetDF() -> None:
    global stateList
    out.clear_output()
    with out:
        fluorophores, solvents, methods = r.fluorophores_solvents_methods()
        if spectra_widg.value:
            make_multindex(solvents + [r.Solvents.meoh],
                        [
                                [i for i in r.Fluorophores if i.ref or i.revised],
                                list(r.spectraType)
                        ], 
                        ['Fluorophore', 'Spectrum'],
                        'spectra', 
                        None, object)

        if energy_widg.value:
            make_multindex(solvents + [r.Solvents.gas],
                        [
                                [i for i in r.Fluorophores if i.gas or i.revised], 
                                ['a', 'a_g', 'e', 'e_g', 'zz', 'zz_g', 'qy', 'fl', 'conc']
                        ], 
                        ['Fluorophore', 'Energy'],
                        'dataset', 
                        0.0, float)
                        

        if progress_widg.value:
            make_multindex(solvents + [r.Solvents.gas], 
                        [
                                [i for i in r.Fluorophores if i.gas or i.revised], 
                                [r.States.s0, r.States.s1, r.States.s2], 
                                [i for i in r.MetaJobs if i.used]
                        ], 
                        ['Fluorophore', 'State', 'MetaJob'],
                        'progress', 
                        None, object)
            stateList = [r.States.s0, r.States.s1, r.States.s2]
            

        if comp_freq_widg.value:
            make_multindex(solvents + [r.Solvents.gas], 
                        [
                                [i for i in r.Fluorophores if i.gas or i.revised], 
                                [r.States.s0, r.States.s1, r.States.s2], 
                                [i for i in r.MetaJobs if i.job == r.Jobs.freq and i.used],
                                [i for i in r.Energy.Freq]
                        ], 
                        ['Fluorophore', 'State', 'MetaJob', 'Property'],
                        'comp-freq', 
                        0.0, float)

        if comp_em_widg.value:
            make_multindex(solvents + [r.Solvents.gas], 
                        [
                                [i for i in r.Fluorophores if i.gas or i.revised], 
                                [r.States.s0, r.States.s1, r.States.s2], 
                                [i for i in r.MetaJobs if i.job == r.Jobs.em and i.used],
                                [i for i in r.Energy.Emission]
                        ], 
                        ['Fluorophore', 'State', 'MetaJob', 'Property'],
                        'comp-em', 
                        0.0, float)
            

        if comp_ex_widg.value:
            make_multindex(solvents + [r.Solvents.gas], 
                        [
                                [i for i in r.Fluorophores if i.gas or i.revised], 
                                [r.States.s0, r.States.s1, r.States.s2], 
                                [i for i in r.MetaJobs if i.job == r.Jobs.ex and i.used],
                                [i for i in r.Energy.Excitation]
                        ], 
                        ['Fluorophore', 'State', 'MetaJob', 'Property'],
                        'comp-ex', 
                        0.0, float)
            
        if comp_casscf_widg.value:
            make_multindex(solvents + [r.Solvents.gas], 
                        [
                                [i for i in r.Fluorophores if i.gas or i.revised], 
                                [r.States.s0, r.States.s1, r.States.s2], 
                                [i for i in r.MetaJobs if i.job in [r.Jobs.casscf, r.Jobs.casscfOpt] and i.used],
                                [i for i in r.Energy.CASSCF]
                        ], 
                        ['Fluorophore', 'State', 'MetaJob', 'Property'],
                        'comp-casscf', 
                        0.0, float)

    if displayDF_widg.value and (spectra_widg.value or 
                                 energy_widg.value or 
                                 comp_ex_widg.value or
                                 comp_em_widg.value or
                                 comp_freq_widg.value or
                                 comp_casscf_widg.value
                                ):
        prettyDisplay()

    if displayDF_widg.value and progress_widg.value:
        prettyDisplay(full=True)
        
def timedOut() -> None:
    out.clear_output()
    with out:
        with r.monarchHandler() as mon:
            mon.timed_out()
            print('\n\n')

def viewAndPull():
    with r.monarchHandler() as mon:
        check_progress(mon)

def showDF() -> None:
    out.clear_output()
    prettyDisplay()

def showAllProgress() -> None:
    out.clear_output()
    prog, spec, en = (progress_widg.value, spectra_widg.value, energy_widg.value)
    progress_widg.value, spectra_widg.value, energy_widg.value = (True, False, False)
    prettyDisplay(full=True)
    progress_widg.value, spectra_widg.value, energy_widg.value = (prog, spec, en )

def showAll() -> None:
    out.clear_output()
    prog, spec, en = (progress_widg.value, spectra_widg.value, energy_widg.value)
    progress_widg.value, spectra_widg.value, energy_widg.value = (True, True, True)
    prettyDisplay(full=True)
    progress_widg.value, spectra_widg.value, energy_widg.value = (prog, spec, en)

# Input handler
def readWidg(s0_in, s1_in, s2_in, progress_in, spectra_in, energy_in, comp_freq_widg_in, comp_ex_widg_in, comp_em_widg_in, comp_casscf_widg_in) -> None:
    global stateList
    global dbList
    stateList = []
    dbList = []

    if s0_in: stateList            += [r.States.s0]
    if s1_in: stateList            += [r.States.s1]
    if s2_in: stateList            += [r.States.s2]
    if progress_in: dbList         += ['progress']
    if spectra_in: dbList          += ['spectra']
    if energy_in: dbList           += ['dataset']
    if comp_freq_widg_in: dbList   += ['comp-freq']
    if comp_ex_widg_in: dbList     += ['comp-em']
    if comp_em_widg_in: dbList     += ['comp-ex']
    if comp_casscf_widg_in: dbList += ['comp-ex']

def clear():
    out.clear_output()


displayDF_widg = widgets.ToggleButton(value=True, icon='check', description='Show the modified DBs', layout=layout)

progress_widg = widgets.ToggleButton(value=False, icon='check', description='Progress DB')
spectra_widg = widgets.ToggleButton(value=False, icon='check', description='Spectra DB')
energy_widg = widgets.ToggleButton(value=False, icon='check', description='Energy Allcation DB', layout=layout)

comp_freq_widg = widgets.ToggleButton(value=False, icon='check', description='Frequencies DB', layout=layout)
comp_em_widg = widgets.ToggleButton(value=False, icon='check', description='Emission DB', layout=layout)
comp_ex_widg = widgets.ToggleButton(value=False, icon='check', description='Excitation DB', layout=layout)
comp_casscf_widg = widgets.ToggleButton(value=False, icon='check', description='CAS DB', layout=layout)

comp = widgets.HBox([comp_ex_widg, comp_em_widg, comp_freq_widg, comp_casscf_widg])
children = [comp]
comp_widg = widgets.Tab()
comp_widg.children = children
comp_widg.set_title(0, 'Computational DBs')

db_widg = widgets.VBox([widgets.HTML('DB/DF Selection'), widgets.VBox([widgets.HBox([spectra_widg, energy_widg, progress_widg]), comp_widg])])

s0_widg = widgets.ToggleButton(value=True, icon='check', description='S0')
s1_widg = widgets.ToggleButton(value=True, icon='check', description='S1')
s2_widg = widgets.ToggleButton(value=True, icon='check', description='S2')
widg_loader = widgets.interactive_output(readWidg, {'s0_in': s0_widg,
                                                    's1_in': s1_widg,
                                                    's2_in': s2_widg,
                                                    'progress_in': progress_widg,
                                                    'spectra_in': spectra_widg,
                                                    'energy_in': energy_widg,
                                                    'comp_freq_widg_in': comp_freq_widg,
                                                    'comp_ex_widg_in': comp_ex_widg,
                                                    'comp_em_widg_in': comp_em_widg,
                                                    'comp_casscf_widg_in': comp_casscf_widg
                                                    })
states_widg = widgets.VBox([widgets.HTML('State Selection'), widgets.HBox([widg_loader, s0_widg, s1_widg, s2_widg])])

to_widg = widgets.interactive(timedOut, {'manual' : True, 'manual_name' : 'Print Timed-Out Jobs'})
vis_widg = widgets.interactive(visualise_ds, {'manual' : True, 'manual_name' : 'Visualise Dataset'})
viewPull_widg = widgets.interactive(viewAndPull, {'manual' : True, 'manual_name' : 'View and Pull'})
fullCheck_widg = widgets.ToggleButton(value=False, icon='check', description='Force Full Check of Progress', layout=layout)
pull_widg = widgets.VBox([viewPull_widg, fullCheck_widg])
jobs_widg = widgets.SelectMultiple(options=metaJobList, rows=len(metaJobList), description='Which Jobs to Check/Display', value=metaJobList)

showDB_widg = widgets.interactive(showDF, {'manual' : True, 'manual_name' : 'Show Selected Dataframes'})
showALLDB_widg = widgets.interactive(showAllProgress, {'manual' : True, 'manual_name' : 'Show ALL Progress'})
showALL_widg = widgets.interactive(showAll, {'manual' : True, 'manual_name' : 'Show ALL DFs'})
clear_widg = widgets.interactive(clear, {'manual' : True, 'manual_name' : 'Clear Output'})
show_widg = widgets.VBox([widgets.HTML('Displaying Options'), widgets.HBox([showDB_widg, showALL_widg, showALLDB_widg, clear_widg])])

view = widgets.VBox([displayDF_widg, 
                    db_widg, 
                    states_widg, 
                    widgets.VBox([widgets.HTML('Actions'), widgets.HBox([to_widg, vis_widg, pull_widg])]), 
                    jobs_widg, 
                    show_widg])


reset_widg = widgets.interactive(resetDF, {'manual' : True, 'manual_name' : 'Reset DFs'})

reset = widgets.VBox([displayDF_widg, 
                      db_widg, 
                      reset_widg, 
                    #   states_widg,
                    #   show_widg
                      ])

metajob_widg = widgets.Dropdown(options=r.MetaJobs, description='Metajob to Add/Remove', layout=layout)
add_metajob_widg = widgets.interactive(add_metajob, {'manual' : True, 'manual_name' : 'Add MetaJob'})
rem_metajob_widg = widgets.interactive(rem_metajob, {'manual' : True, 'manual_name' : 'Remove Metajob'})
metajob = widgets.VBox([displayDF_widg, 
                     metajob_widg, 
                     widgets.VBox([widgets.HTML('Actions'), widgets.HBox([add_metajob_widg, rem_metajob_widg])]),
                     show_widg])

fluorophore_widg = widgets.Dropdown(options=r.Fluorophores, description='Fluorophore', layout=layout)
add_fluorophore_widg = widgets.interactive(add_fluorophore, {'manual' : True, 'manual_name' : 'Add Fluorophore'})
rem_fluorophore_widg = widgets.interactive(rem_fluorophore, {'manual' : True, 'manual_name' : 'Remove fluorophore'})
fluo = widgets.VBox([displayDF_widg, 
                     fluorophore_widg, 
                     db_widg,
                     widgets.VBox([widgets.HTML('Actions'), widgets.HBox([add_fluorophore_widg, rem_fluorophore_widg])]),
                     show_widg])

solvent_widg = widgets.Dropdown(options=r.Solvents, description='Solvent', layout=layout)
add_solvent_widg = widgets.interactive(add_solvent, {'manual' : True, 'manual_name' : 'Add Solvent'})
rem_solvent_widg = widgets.interactive(rem_solvent, {'manual' : True, 'manual_name' : 'Remove Solvent'})
solv = widgets.VBox([displayDF_widg, 
                     solvent_widg, 
                     db_widg,
                     widgets.VBox([widgets.HTML('Actions'), widgets.HBox([add_solvent_widg, rem_solvent_widg])]),
                     show_widg])

children = [view, reset, metajob, fluo, solv]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'General')
tab.set_title(1, 'Reset DFs')
tab.set_title(2, 'Add/Remove MetaJob')
tab.set_title(3, 'Add/Remove Fluorophores')
tab.set_title(4, 'Add/Remove Solvent')
display(tab)

out = widgets.Output()
display(out)

Output()